In [28]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import base64
import mimetypes
from PIL import Image
from io import BytesIO
import pandas as pd
from io import StringIO
import fitz
load_dotenv()

True

In [7]:
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'))

In [30]:
os.mkdir('images/biedronka20112023-25112023')
doc = fitz.open("images/biedronka20112023-25112023.pdf")

for idx in range(doc.page_count):
    # Select the page you want to convert (0 for the first page)
    page = doc.load_page(idx)

    # Convert it to a pixmap (an image)
    pix = page.get_pixmap()

    # Save the image
    image_path = f"images/biedronka20112023-25112023/page{idx}.png"
    pix.save(image_path)

# Close the document
doc.close()

In [44]:
def chat(query, image_path):
    # Guess the MIME type of the image
    mime_type, _ = mimetypes.guess_type(image_path)
    if not mime_type or not mime_type.startswith('image'):
        raise ValueError("The file type is not recognized as an image")
    
    # Read the image binary data
    with open(image_path, 'rb') as image_file:
        image_data = image_file.read()
        encoded_string = base64.b64encode(image_data).decode('utf-8')
    
    # Format the result with the appropriate prefix
    image_base64 = f"data:{mime_type};base64,{encoded_string}"

    # Read image for size using PIL
    image = Image.open(BytesIO(image_data))
    width, height = image.size
    print("Width:", width, "Height:", height)
    
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": query},
                    {
                        "type": "image_url",
                        "image_url": {"url": image_base64, "detail": "high"}
                    },
                ],
            }
        ],
        max_tokens=300,
    )

    return response.choices[0].message.content

In [45]:
query = ''' You are given a page from a polish grocery shop offer. 
            I need a list of items on this offer, their category, discounts applied and price if it is known.
            Also write the type of the product like: tomato, bun, tortilla, tooth paste, strawberries etc. 
            Return in a format: product/type/discount/price new line. Return only this format and nothing else. If you don't know return: "NA/NA/NA/NA" '''

In [46]:
offer_strings = ''
for idx in range(70):
    response = chat(query , f'images/biedronka20112023-25112023/page{idx}.png')
    offer_strings += response + '\n'

Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 Height: 842
Width: 596 

In [47]:
offer = ''
for row in offer_strings.split('\n'):
    if '/' in row and len(row.split('/')) == 4:
        offer += row + '\n'

print(offer)

Milk UHT 3,2%/milk/42% discount/1.99 PLN
Sugar White/sugar/41% discount/3.39 PLN
Giuseppe Pizza/pizza/second item 70% discount/NA
Chicken drumstick/chicken/41% discount/6.99 PLN
Kiełbasa żywiecka/Zywiec sausage/second item 74% discount/10.99 PLN
Piwo Harnaś/beer/68% discount/2.29 PLN
NA/NA/NA/NA
mleko UHT 3,2%/milk/42% discount when buying 6/1.99 PLN each when buying 6
bułeczki śniadaniowe Pano/bread/1+1 gratis/NA
kosmetyki Gliss/hair care products/1+1 gratis/NA
NA/NA/NA/NA
cukier biały/sugar/41% discount when buying 10/3.39 PLN per 1 kg pack with a card
tortilla/tortilla/2+2 gratis/NA
toothpaste/toothpaste/1+1 gratis/NA
Prince Polo Classic wafers/262.5g/2+2 gratis with card/11.99 PLN
Gillette products/various types/1+1 gratis with card/NA
diapers (size 3, 4-9 kg)/baby care/1+1 gratis with card or app/54 PLN
diapers (size 4, 7-16 kg)/baby care/1+1 gratis with card or app/50 PLN
diapers (size 5, 15-25 kg)/baby care/1+1 gratis with card or app/44 PLN
diapers (size 6, 16+ kg)/baby care/1+

In [51]:
# Using StringIO to simulate a file-like object
string_io = StringIO(offer.strip())

# Reading the string into a pandas DataFrame
df = pd.read_csv(string_io, sep="/", names=["Product", "Type", "Discount", "Price"])

df.to_csv('images/biedronka20112023-25112023/result.csv')